In [4]:
import vk
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm, tqdm_pandas

In [5]:
# opennig vk session with access token.
session = vk.Session(access_token='581cab69a93dd42efd81513b12b83acab2fb9593d163d4aa1d5f8de7df197443416e5cd4e87eb78324274')
api = vk.API(session)
# example use with id, my id
#api.users.get(user_ids='architectofreality' )

In [6]:
#searching for a keyword glaukoma starting 48 hours ago, max 200 counts can be scrapped at once
glaukoma=api.newsfeed.search(q='glaukoma',count=200, start_time=48)

### get last 200 posts --> take the time of the last post --> at the next iteration set the end time = time_last_post-1. Set the start time to 1 

In [4]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

Wall time: 228 ms


In [5]:
# dropping the first element and creat a dataframe with the rest
df=pd.DataFrame(glaukoma_rus[1:])

In [6]:
#getting info of each user, deactivated column
ids=pd.DataFrame(api.users.get(user_ids=abs(df['owner_id'])))
ids.head()

,deactivated,first_name,last_name,uid
0,deleted,Elena,Zatylkina,70740559


In [7]:
def active_users_info(x):
    # x is api newsfeed search result
    # use each user only once to get personal info (different posts from same user have the same personal info)
    unique_users = list(set(x['owner_id'].astype(int)))
    users = pd.DataFrame(api.users.get(user_ids=unique_users,fields = ['occupation','education',
                                                                       'universities','bdate',
                                                                       'city','interests',
                                                                       'activities','site']))

    try:     
        # fill active users status with valid
        users['deactivated'].fillna('Valid',inplace = True) 
        # select all deactivated user ids
        deactivated_ids = list(users[users['deactivated']!='Valid'].uid)

        # select all active user posts
        active_users = x[-x.owner_id.isin(deactivated_ids)]
        active_users_info = users[users['deactivated']=='Valid']
    except:
        active_users = x  
        active_users_info = users

    # merge post info and user info
    overall_data = active_users.merge(active_users_info, left_on='owner_id', right_on='uid')
    return overall_data

In [8]:
%%time
key_words = ['#глаза',
             '#глазки', '#зрение', '#лечениезрения', '#детскоезрение', '#лечениеглаз',
             '#офтальмолог','#офтальмология', '#офтальмохирург', '#офтальмохирургия', '#детскоезрение',
             '#детскийофтальмолог', '#офтальмологмосква','#дакриоцистит', '#дакриоциститноворожденных',
             '#дакриоциститмассаж','#дакриоциститлечение', '#дакриоцисториностомия', '#зондированиеносослёзногоканала',
             '#зондированиеноворождённых','#нистагм', '#лечениенистагма','#нистагмоперация', '#нистагмъ',
             '#близорукость', '#миопия', '#лечениеблизорукости', '#близорукостьнеприговор',
             '#близоркостьлечение', '#близорукостьонатакая', '#близорукостьнепорок',
             '#близорукостьудетей','#дальнозоркость', '#гиперметропия', '#лечениедальнозоркости',
             '#дальнозоркостьлечение','#дальнозоркостьудетей', '#дальнозоркости','#косоглазие',
             '#страбизм', '#лечениекосоглазия', '#косоглазиелечение', '#косоглазиеэтосексуально',
             '#косоглазиебишкек', '#косоглазиенепорок', '#косоглазие дети', '#косоглазиеэтомодно',
             '#астигматизм', '#астигматизмлечение', '#астигматизмудетей','#амблиопия', '#амблиопиявысокойстепени',
             '#амблиопиямосква', '#катаракта','#катарактавладивосток', '#катарактачелябинск',
             '#катаракта лечение москва','#катарактаоперация', '#глаукома', '#глаукомалечение', 
             '#глаукомавладивосток','#глаукомачелябинск', '#птоз', '#птозверхнеговека', '#птозвека',
             '#птозвек', '#птозанет','#лазернаякоррекция', '#лазернаякоррекциязрения', '#лазернаякоррекциязрениямосква',
             '#лазернаякоррекцияакция', '#лазернаякоррекциямосква', '#лазернаякоррекциязренияомикрон',
             '#лазернаякоррекцияскидки', '#lasik', '#фрк', '#ласик', '#ласикмосква', '#ячмень', '#халязион']
desired_time = 86400*365 # how much back we want to go in seconds
entire_df = pd.DataFrame()
# Overall iteration count
k = 1
for key_word in key_words:
    print('search for keyword: ', key_word)
    date = time.time() # now
    excess_time = True
    all_df = pd.DataFrame()
    # Iteration count for each key word
    c=1
    # Iterator for error count (TimeOut Error)
    error = 0
    while excess_time:
        # in case a ReadTimeOut Error occurs
        try:
            # Print which iteration it is for current key word   
            print(c,'iteration')

            '''
            # Resting each overall 60 iterations
            if k%60 == 0:
                print('Rest for 10 min')
                time.sleep(60*10)
                print('Work')
            '''

            # print iteration count, extended tells us group vs user
            newsfeed_search_results = api.newsfeed.search(q=[key_word],count=200, end_time = date-1, start_time=1,extended=1)
            nsr_df = pd.DataFrame(newsfeed_search_results[1:])
            # Filter posts done by groups
            try:
                nsr_df = nsr_df[nsr_df.owner_id>0].reset_index()
            except:
                pass
            
            if len(nsr_df)>0: # check if g contains data
                new_data = active_users_info(nsr_df)
                all_df = pd.concat([all_df,new_data])
                date=newsfeed_search_results[-1]['date']
                time.sleep(31) # Sleep time to not exceed requests of the same method

            elif len(nsr_df) <= 0 and c == 1: # check if there is data
                print('No posts with that keyword(s)!')
                time.sleep(31) # Sleep time to not exceed requests of the same method
                break

            elif len(nsr_df) <= 0 and c != 1: # check if there are posts eft to scrape
                print('No more posts to scrape!')
                time.sleep(31) # Sleep time to not exceed requests of the same method
                break

            # increment the iteration counts
            c+=1
            k+=1

            if date<time.time()-desired_time: # check termination condition
                excess_time = False
        # just repeat the request
        except:
            print('Timeout Error: sleep for 2 min')
            time.sleep(60*2)
            error += 1
            if error == 2:
                print('Encountered TimeOut Error 2 times')
                break
            else:
                continue
        
    all_df['keyword'] = key_word
    print('all_df length: ',len(all_df))
    entire_df = pd.concat([entire_df,all_df])
    

# getting list of cities by their ids
all_city_ids = entire_df.city.unique().astype(int)
city_name = []
# cant scrape more than 1000 in 1 request
for cities in range(int(len(all_city_ids)/1000)+1):
    city = api.database.getCitiesById(city_ids = all_city_ids[cities*1000:(cities+1)*1000])
    city_name = city_name+city

# creating dataframe from the list of the cities    
entire_df['city'] = entire_df['city'].astype(int)
# merging 2 dataframes on city id
entire_df = entire_df.merge(pd.DataFrame(city_name), left_on='city', right_on='cid')

print('entire_df length: ',len(entire_df))

search for keyword:  #глаза
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
11 iteration
12 iteration
13 iteration
14 iteration
15 iteration
16 iteration
17 iteration
18 iteration
19 iteration
20 iteration
21 iteration
22 iteration
23 iteration
24 iteration
25 iteration
26 iteration
27 iteration
28 iteration
29 iteration
30 iteration
31 iteration
32 iteration
33 iteration
34 iteration
35 iteration
36 iteration
37 iteration
38 iteration
39 iteration
40 iteration
41 iteration
42 iteration
43 iteration
44 iteration
45 iteration
46 iteration
47 iteration
48 iteration
49 iteration
50 iteration
51 iteration
52 iteration
53 iteration
54 iteration
55 iteration
56 iteration
57 iteration
58 iteration
59 iteration
60 iteration
61 iteration
62 iteration
63 iteration
64 iteration
65 iteration
66 iteration
67 iteration
68 iteration
69 iteration
70 iteration
71 iteration
72 iteration
73 iteration
74 iteration
75 iteration
76 ite

all_df length:  1
search for keyword:  #астигматизмудетей
1 iteration
No posts with that keyword(s)!
all_df length:  0
search for keyword:  #амблиопия
1 iteration
all_df length:  36
search for keyword:  #амблиопиявысокойстепени
1 iteration
No posts with that keyword(s)!
all_df length:  0
search for keyword:  #амблиопиямосква
1 iteration
No posts with that keyword(s)!
all_df length:  0
search for keyword:  #катаракта
1 iteration
2 iteration
3 iteration
4 iteration
all_df length:  133
search for keyword:  #катарактавладивосток
1 iteration
No posts with that keyword(s)!
all_df length:  0
search for keyword:  #катарактачелябинск
1 iteration
No posts with that keyword(s)!
all_df length:  0
search for keyword:  #катаракта лечение москва
1 iteration
all_df length:  7
search for keyword:  #катарактаоперация
1 iteration
No posts with that keyword(s)!
all_df length:  0
search for keyword:  #глаукома
1 iteration
2 iteration
3 iteration
all_df length:  119
search for keyword:  #глаукомалечение
1 i

In [124]:
pd.options.display.max_columns = 50
entire_df.head()

,activities,attachment,attachments,bdate,city,comments,copy_owner_id,copy_post_date,copy_post_id,copy_post_type,copy_text,date,education_form,education_status,faculty,faculty_name,first_name,from_id,geo,graduation,group,id,index,interests,keyword,last_name,likes,marked_as_ads,occupation,owner_id,post_id,post_source,post_type,reposts,signer_id,site,text,uid,universities,university,university_name,user
0,,"{'type': 'photo', 'photo': {'pid': 456240824, ...","[{'type': 'photo', 'photo': {'pid': 456240824,...",5.12.1984,35,"{'count': 0, 'groups_can_post': True, 'can_pos...",NaN,NaN,NaN,NaN,NaN,1.518175e+09,Distance Learning,NaN,29628.0,Факультет экономики и управления,Tatyana,4458562.0,NaN,2010.0,NaN,3056.0,0.0,,#глаза,Demura,"{'count': 1, 'user_likes': 0, 'can_like': 1, '...",NaN,"{'type': 'work', 'name': 'ГУ-УПФ РФ'}",4458562.0,NaN,"{'type': 'api', 'platform': 'android'}",post,"{'count': 0, 'user_reposted': 0}",NaN,,❄Свободное время для процедуры Ламинирование р...,4458562.0,"[{'id': 16639, 'country': 1, 'city': 35, 'name...",16639.0,СПбГУСЭ (НФ),"{'uid': 4458562, 'first_name': 'Tatyana', 'las..."
1,,"{'type': 'photo', 'photo': {'pid': 456239675, ...","[{'type': 'photo', 'photo': {'pid': 456239675,...",10.2.1980,483,"{'count': 0, 'groups_can_post': True, 'can_pos...",NaN,NaN,NaN,NaN,NaN,1.518174e+09,NaN,NaN,0.0,,Ekaterina,226768214.0,NaN,0.0,NaN,796.0,1.0,,#глаза,Poznyakova,"{'count': 1, 'user_likes': 0, 'can_like': 1, '...",NaN,NaN,226768214.0,NaN,"{'type': 'api', 'platform': 'instagram', 'url'...",post,"{'count': 0, 'user_reposted': 0}",NaN,,#митино#солнце#🌞#в#глаза😜,226768214.0,[],0.0,,"{'uid': 226768214, 'first_name': 'Ekaterina', ..."
2,,"{'type': 'photo', 'photo': {'pid': 456240889, ...","[{'type': 'photo', 'photo': {'pid': 456240889,...",21.4.1990,314,"{'count': 0, 'groups_can_post': True, 'can_pos...",NaN,NaN,NaN,NaN,NaN,1.518174e+09,NaN,NaN,0.0,,Nadya,457471680.0,NaN,0.0,NaN,1480.0,4.0,,#глаза,Dorofeeva,"{'count': 3, 'user_likes': 0, 'can_like': 1, '...",NaN,NaN,457471680.0,NaN,{'type': 'mvk'},post,"{'count': 0, 'user_reposted': 0}",NaN,,Твои #глаза мои #мысли...❤Не ревнуй меня прошу...,457471680.0,[],0.0,,"{'uid': 457471680, 'first_name': 'Nadya', 'las..."
3,NaN,"{'type': 'photo', 'photo': {'pid': 456240430, ...","[{'type': 'photo', 'photo': {'pid': 456240430,...",14.7.1996,1710959,"{'count': 0, 'groups_can_post': True, 'can_pos...",NaN,NaN,NaN,NaN,NaN,1.518173e+09,NaN,NaN,NaN,NaN,Kristina,182669526.0,NaN,NaN,NaN,18597.0,6.0,NaN,#глаза,Osievskaya,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...",NaN,"{'type': 'university', 'id': 90312, 'name': 'U...",182669526.0,NaN,"{'type': 'api', 'platform': 'instagram', 'url'...",post,"{'count': 0, 'user_reposted': 0}",NaN,,Приватизировала игрушку-подушку. #кошка #чёрна...,182669526.0,NaN,NaN,NaN,"{'uid': 182669526, 'first_name': 'Kristina', '..."
4,"No Access(работа), Недовидеомэйкинг","{'type': 'photo', 'photo': {'pid': 456241482, ...","[{'type': 'photo', 'photo': {'pid': 456241482,...",3.1.1992,2,"{'count': 1, 'groups_can_post': True, 'can_pos...",NaN,NaN,NaN,NaN,NaN,1.518171e+09,Distance Learning,Alumnus (Bachelor's),135.0,Институт технологических машин и транспорта ле...,Eduard,2507921.0,NaN,2015.0,NaN,7238.0,9.0,"Недоблоггинг, попытки чего то добиться. Web de...",#глаза,Gofman,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...",NaN,"{'type': 'work', 'id': 3785, 'name': 'МегаФон'}",2507921.0,NaN,"{'type': 'api', 'platform': 'instagram', 'url'...",post,"{'count': 0, 'user_reposted': 0}",NaN,stihi.ru/avtor/genslife | ask.fm/genslife ...,Старые травмы снова себя проявили((<br>Ведь та...,2507921.0,"[{'id': 40, 'country': 1, 'city': 2, 'name': '...",40.0,СПбГЛТУ им. Кирова,"{'uid': 2507921, 'first_name': 'Eduard', 'last..."


In [144]:
entire_df.signer_id.value_counts()

663391.0       2
401262553.0    1
25036294.0     1
50487342.0     1
304093903.0    1
5017862.0      1
89230156.0     1
1331895.0      1
6115312.0      1
425103394.0    1
162342563.0    1
26673534.0     1
13354142.0     1
19402428.0     1
1003437.0      1
165679842.0    1
125042986.0    1
75656136.0     1
Name: signer_id, dtype: int64

In [140]:
def type_scrap(x):
    try:
        return x['type']
    except:
        return None
entire_df.occupation.apply(type_scrap).unique()

array(['work', None, 'university', 'school'], dtype=object)

In [105]:
city1=api.database.getCitiesById(city_ids=entire_df.city.unique().astype(int)[:999])
city2=api.database.getCitiesById(city_ids=entire_df.city.unique().astype(int)[999:])
city_name=city1+city2

In [114]:
city_name

[{'cid': 35, 'name': 'Velikiy Novgorod'},
 {'cid': 483, 'name': 'Krasnogorsk'},
 {'cid': 314, 'name': 'Kyiv'},
 {'cid': 1710959, 'name': 'Chişinău '},
 {'cid': 2, 'name': 'Saint Petersburg'},
 {'cid': 1, 'name': 'Moscow'},
 {'cid': 147, 'name': 'Tyumen'},
 {'cid': 148, 'name': 'Ulan-Ude'},
 {'cid': 116, 'name': 'Pyatigorsk'},
 {'cid': 627, 'name': 'Simferopol'},
 {'cid': 56, 'name': 'Izhevsk'},
 {'cid': 25, 'name': 'Barnaul'},
 {'cid': 26, 'name': 'Belgorod'},
 {'cid': 75, 'name': 'Kursk'},
 {'cid': 282, 'name': 'Minsk'},
 {'cid': 20695, 'name': 'Bucha'},
 {'cid': 1044071, 'name': 'Ponyri'},
 {'cid': 123, 'name': 'Samara'},
 {'cid': 650, 'name': 'Dnipropetrovsk (Dnipro)'},
 {'cid': 141, 'name': 'Tver'},
 {'cid': 2642, 'name': 'Cherkassy'},
 {'cid': 22, 'name': 'Arkhangelsk'},
 {'cid': 124, 'name': 'Saransk'},
 {'cid': 82, 'name': 'Magnitogorsk'},
 {'cid': 90, 'name': 'Nalchik'},
 {'cid': 158, 'name': 'Chelyabinsk'},
 {'cid': 146, 'name': 'Tula'},
 {'cid': 33, 'name': 'Bryansk'},
 {'cid

In [115]:
pd.DataFrame(city_name)

,cid,name
0,35,Velikiy Novgorod
1,483,Krasnogorsk
2,314,Kyiv
3,1710959,Chişinău
4,2,Saint Petersburg
5,1,Moscow
6,147,Tyumen
7,148,Ulan-Ude
8,116,Pyatigorsk
9,627,Simferopol


In [117]:
entire_df['city'] = entire_df['city'].astype(int)
newer_df = entire_df.merge(pd.DataFrame(city_name), left_on='city', right_on='cid')

In [121]:
newer_df.name.value_counts()

Moscow                     3040
Saint Petersburg           1511
Krasnodar                   785
Kyiv                        530
Novosibirsk                 517
Minsk                       362
Krasnoyarsk                 348
Odessa                      333
Yekaterinburg               332
Kazan                       309
Chelyabinsk                 281
Rostov-on-Don               271
Voronezh                    265
Volgograd                   218
Kharkiv                     215
Ufa                         208
Omsk                        206
Nizhny Novgorod             205
Almaty                      202
Samara                      199
Tyumen                      194
Izhevsk                     164
Saratov                     156
Perm                        146
Naberezhnye Chelny          143
Dnipropetrovsk (Dnipro)     127
Irkutsk                     115
Barnaul                     111
Tolyatti                    109
Kursk                       108
                           ... 
Berchtes

In [9]:
entire_df.to_csv(r'D:\vk_all_keywors_365_days_vk8_10Feb.csv')

In [11]:
%%time
overall_data=entire_df.copy()
# creating a dataframe for dictionary values
from tqdm import tqdm
new_df = pd.DataFrame(columns = ['post_id','user_id','post','sex','Name_Surname',
                                 'likes','comments','reposts','platform','date','key_word',
                                 'city','university','faculty_name',
                                 'eduaction_status','activities','interests','work_type',
                                 'work_place','site','bdate'])
# resetting the index it make circles of 0-200
#active_users.reset_index(inplace = True)
    
# making function to apply to get platform
def platform(x):
    try:
        return x.post_source['type'] + '_' + x.post_source['platform']
    except:
        return x.post_source['type']

# making function to get work type    
def type_scrap(x):
    try:
        return x['type']
    except:
        return None
    
# making function to get work place   
def place_scrap(x):
    try:
        return x['name']
    except:
        return None

    
new_df['post_id'] = overall_data['id']
new_df['platform'] = overall_data.apply(platform, axis = 1)
new_df['key_word'] = overall_data['keyword']
new_df['user_id'] = overall_data['user'].apply(lambda x: x['uid'])   
new_df['Name_Surname'] = overall_data['user'].apply(lambda x: x['first_name'])+'_'+overall_data['user'].apply(lambda x: x['last_name'])
new_df['post'] = overall_data['text']
new_df['likes_count'] = overall_data['likes'].apply(lambda x: x['count'])
new_df['comments_count'] = overall_data['comments'].apply(lambda x: x['count']) 
new_df['reposts_count'] = overall_data['reposts'].apply(lambda x: x['count'])
new_df['date'] = overall_data['date'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
new_df['sex'] = overall_data['user'].apply(lambda x: 'Female' if x['sex']==1 else 'Male' if x['sex']==2  else 'Not Specified')
new_df['city'] = overall_data['name']
new_df['university'] = overall_data['university_name']
new_df['gradutaion_year'] = overall_data['graduation'].astype(int)
new_df['faculty_name'] = overall_data['faculty_name']
new_df['education_status'] = overall_data['education_status']
new_df['activities'] = overall_data['activities']
new_df['interests'] = overall_data['interests']
new_df['work_type'] = overall_data['occupation'].apply(type_scrap)
new_df['work_place'] = overall_data['occupation'].apply(place_scrap)
new_df['site'] = overall_data['site']
new_df['bdate'] = overall_data['bdate']
#new_df.head()

Wall time: 1.13 s


In [14]:
new_df.to_csv(r'D:\vk_all_keywors_365_days_vk8_10Feb_pretty.csv')

In [91]:
new_df.iloc[2].post

'Твои #глаза мои #мысли...❤Не ревнуй меня прошу,останься этой ночью у меня))И наши с тобой сердца будут всегда с нами!💘💘'

In [ ]:
# getting the comments of posts
def commentscrapper(x):
    uid = x['user_id']
    key_word = x['key_word']
    time.sleep(1)
    post = api.wall.search(owner_id = uid, query = key_word)
    time.sleep(1)
    if post[0] == 1:
        pid = post[1]['id']
        likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
        time.sleep(1)
        reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
        time.sleep(1)
        try:           
            coms = api.wall.getComments(owner_id = uid, post_id = pid, need_likes = 1)
            time.sleep(1)
            return {'comments':[post[0],coms[1:]],'users_liked': likes,'users_reposted': reposts}
        except:
            return {'comments':[post[0],'Access Denied'],'users_liked':likes,'users_reposted': reposts}
    elif post[0]==0:
        return {'comments':'post not found','users_liked':'post not found','users_reposted': 'post not found'}
    else:
        try:
            my_date = x['date']
            post_dates = [datetime.fromtimestamp(y['date']).strftime('%Y-%m-%d %H:%M:%S') for y in post[1:]]
            index = post_dates.index(my_date)
            try:
                pid = post[index+1]['id']
                likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
                time.sleep(1)
                reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
                time.sleep(1)
                coms = api.wall.getComments(owner_id = uid, post_id = pid, need_likes = 1)
                time.sleep(1)
                return {'comments':[post[0],coms[1:]],'users_liked':likes,'users_reposted': reposts}
            except:
                pid = post[index+1]['id']
                likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
                time.sleep(1)
                reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
                time.sleep(1)
                return {'comments':[post[0],'Access Denied'],'users_liked':likes,'users_reposted': reposts}
        except:
            return {'comments':'time not in Index','users_liked':'time not in Index','users_reposted': 'time not in Index'}
    time.sleep(2)
    

In [8]:
# getting the comments of posts
def commentscrapper(x):
    uid = x['user_id']
    pid = x['post_id']
    likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
    time.sleep(2)
    reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
    time.sleep(2)
    try:           
        coms = api.wall.getComments(owner_id = uid, post_id = pid, need_likes = 1)
        time.sleep(2)
        return {'comments':coms[1:],'users_liked': likes,'users_reposted': reposts,'user_id': uid, 'post_id':pid}
    except:
        return {'comments':'Access Denied','users_liked':likes,'users_reposted': reposts,'user_id': uid, 'post_id':pid}
        time.sleep(2)
    #elif post[0]==0:
        #return {'comments':'post not found','users_liked':'post not found','users_reposted': 'post not found'}
    

In [10]:
int(new_df.shape[0])

22902

In [72]:
clr_data2 = new_df.head().copy()#pd.DataFrame()
for i in tqdm(range(int(new_df.head().shape[0]))):
    try:
        more_new_df = new_df.iloc[i:(i+1)].apply(commentscrapper,axis = 1)
        comments_likes_reposts = more_new_df.apply(pd.Series)
        clr_data2 = pd.concat([clr_data2,comments_likes_reposts],axis=1)
    except:
        print('Timeout Error: sleep for 2 min')
        time.sleep(2)
        continue
#final_data = pd.concat([new_df,clr_data],axis = 1)






  0%|                                                                                            | 0/5 [00:00<?, ?it/s]




 20%|████████████████▊                                                                   | 1/5 [00:09<00:37,  9.31s/it]




 40%|█████████████████████████████████▌                                                  | 2/5 [00:18<00:27,  9.29s/it]




 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:27<00:18,  9.28s/it]




 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:34<00:08,  8.53s/it]




100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:43<00:00,  8.69s/it]






In [16]:
493+578

1071

In [7]:
new_df = pd.read_csv(r'to_Levon_byLevon.csv')

In [14]:
new_df

,Unnamed: 0,post_id,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,...,interests,work_type,work_place,site,bdate,likes_count,comments_count,reposts_count,gradutaion_year,education_status
0,0,3056.0,4458562,❄Свободное время для процедуры Ламинирование р...,Female,Tatyana_Demura,NaN,NaN,NaN,api_android,...,NaN,work,ГУ-УПФ РФ,NaN,5.12.1984,1,0,0,2010.0,NaN
1,1,796.0,226768214,#митино#солнце#🌞#в#глаза😜,Female,Ekaterina_Poznyakova,NaN,NaN,NaN,api_instagram,...,NaN,NaN,NaN,NaN,10.2.1980,1,0,0,0.0,NaN
2,2,1480.0,457471680,Твои #глаза мои #мысли...❤Не ревнуй меня прошу...,Female,Nadya_Dorofeeva,NaN,NaN,NaN,mvk,...,NaN,NaN,NaN,NaN,21.4.1990,3,0,0,0.0,NaN
3,3,18597.0,182669526,Приватизировала игрушку-подушку. #кошка #чёрна...,Female,Kristina_Osievskaya,NaN,NaN,NaN,api_instagram,...,NaN,university,Universitatea Cooperatist-Comerciala din Moldo...,NaN,14.7.1996,0,0,0,NaN,NaN
4,4,7238.0,2507921,Старые травмы снова себя проявили((<br>Ведь та...,Male,Eduard_Gofman,NaN,NaN,NaN,api_instagram,...,"Недоблоггинг, попытки чего то добиться. Web de...",work,МегаФон,stihi.ru/avtor/genslife | ask.fm/genslife ...,3.1.1992,0,1,0,2015.0,Alumnus (Bachelor's)
5,5,1157.0,6625983,Быстрая песня на пять восьмых. Сложно. Страшно...,Male,Anatoly_Kozhaev,NaN,NaN,NaN,api_instagram,...,NaN,university,РАМ им. Гнесиных,NaN,NaN,9,0,0,2012.0,NaN
6,6,2759.0,18092728,Просто почувствуй себя красивой и желанной!😍🤘<...,Female,Natalya_Voltornist,NaN,NaN,NaN,api_instagram,...,NaN,university,ТюмГУ,NaN,7.12,7,0,0,2005.0,Alumna (Specialist)
7,7,4872.0,192590067,#глаза👸🌋≈тамрекилюбви≈тамбеда,Female,Stanislava_Mirnaya,NaN,NaN,NaN,api_android,...,NaN,university,ВСГУТУ (бывш. ВСГТУ),sprashivai.ru/anastysiya_kykla,21.3.1997,15,0,0,NaN,NaN
8,8,109.0,409278702,Ваши любимые палетки МАС ❤️<br>В палетку входи...,Female,Margarita_Tarasova,NaN,NaN,NaN,api_iphone,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,NaN
9,9,49.0,174999924,Видимо не зря в большинстве скульптур изобража...,Male,Vladimir_Zhirkin,NaN,NaN,NaN,api_instagram,...,NaN,work,КЛАССИЧЕСКИЕ БЕСЕДЫ,http://verafarm.ru/,10.5.1980,4,0,0,NaN,NaN


In [12]:
clr_data11 = new_df.copy()#pd.DataFrame()
clr_data11['comments'] = None
clr_data11['users_liked'] = None
clr_data11['users_reposted'] = None
for i in tqdm(range(21460, new_df.shape[0])):
    clr = commentscrapper(new_df.iloc[i])
    clr_data11.set_value(i,'comments' ,clr['comments'] )               
    clr_data11.set_value(i,'users_liked', clr['users_liked'])          
    clr_data11.set_value(i,'users_reposted', clr['users_reposted'])    

#final_data = pd.concat([new_df,clr_data],axis = 1)


  0%|                                                                                         | 0/1442 [00:00<?, ?it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\users\levon.yeghiazaryan\appdata\local\continuum\anaconda3\envs\vk\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\levon.yeghiazaryan\appdata\local\continuum\anaconda3\envs\vk\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "c:\users\levon.yeghiazaryan\appdata\local\continuum\anaconda3\envs\vk\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

c:\users\levon.yeghiazaryan\appdata\local\continuum\anaconda3\envs\vk\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
c:\users\levon.yeghiazaryan\appdata\local\conti

In [74]:
new_df.iloc[20543]

Unnamed: 0                                                          4
post_id                                                          3177
user_id                                                     349341363
post                Как воспринимают цвета дети!<br>Ученые выяснил...
sex                                                              Male
Name_Surname                                          Alexander_Boyko
likes                                                             NaN
comments                                                          NaN
reposts                                                           NaN
platform                                                           vk
date                                              2017-06-08 14:10:08
key_word                                               #офтальмология
city                                                              NaN
university                                                     КубГМУ
graduation_year     

In [51]:
clr_data4.loc[13868:16874]#.drop(index=13869)

,Unnamed: 0,post_id,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,...,work_place,site,bdate,likes_count,comments_count,reposts_count,gradutaion_year,education_status,users_liked,users_reposted
13868,40,577.0,247089921,#кот#идиот#хах#эти#глаза#убивают,Male,Vladimir_Marinich,NaN,None,NaN,api_instagram,...,NaN,NaN,21.3,14,0,0,NaN,NaN,None,None
13869,41,121.0,15887560,#глаза забыл:)),Male,Sanyok_Zarubin,NaN,None,NaN,vk,...,The SLOE [Альтернативный рок],https://vk.com/thesloe,19.7,1,0,0,NaN,NaN,None,None
13870,42,2429.0,50127289,#глаз #глазищи #глаза #зеленыеглаза #животные ...,Female,Valeria_Palousova,NaN,Access Denied,NaN,api_instagram,...,УПРГЖ,https://vk.com/public58712415,NaN,6,0,0,NaN,NaN,"{'count': 6, 'users': [3202435, 5376066, 58324...","{'count': 0, 'users': []}"
13871,43,5685.0,17544846,#вофисе #красотки #работаем #глаза #косички #х...,Female,Anastasia_Stepanova,NaN,Access Denied,NaN,api_instagram,...,ЧГУ им. Ульянова,nastik_77777@mail.ru,27.9.1990,26,0,0,NaN,NaN,"{'count': 26, 'users': [462403, 6407922, 11144...","{'count': 0, 'users': []}"
13872,44,3823.0,39762608,Очі)<br>#Глаза #Очі #folowme @ Czempiela,Male,Olexandr_Konelsky,NaN,"[{'cid': 3824, 'uid': 187020100, 'from_id': 18...",NaN,api_instagram,...,Autoneum,http://sprashivai.ru/Sashokbest100500,26.11.1992,36,2,0,2012.0,NaN,"{'count': 36, 'users': [37130171, 49293896, 56...","{'count': 0, 'users': []}"
13873,45,261.0,343773776,"Ламинирование ресниц - SPA процедура, которая ...",Female,Alyona_Sergienko,NaN,Access Denied,NaN,api_instagram,...,"м. Автозаводская, ул. Трофимова, 2,1",https://www.instagram.com/sergienkoinfo/,NaN,7,0,0,0.0,NaN,"{'count': 7, 'users': [2443398, 13254028, 1394...","{'count': 0, 'users': []}"
13874,46,3278.0,30226830,"Прочитала, что оказывается, карие глаза на сам...",Female,Kristina_Sergienko,NaN,Access Denied,NaN,api_instagram,...,НГПУ,нет,28.5.1987,32,0,0,NaN,NaN,"{'count': 32, 'users': [17576251, 35526120, 37...","{'count': 0, 'users': []}"
13875,47,2503.0,22566467,⚡Мои новиночки⚡<br>.<br>Купила себе замечатель...,Female,Viktoria_Voloschuk,NaN,Access Denied,NaN,api_instagram,...,Атамановка,http://.......ru,15.10,0,0,0,NaN,NaN,"{'count': 0, 'users': []}","{'count': 0, 'users': []}"
13876,48,2872.0,149030765,ДЭНАС очки проблемы со зрением очень актуальны...,Female,Elena_Gureeva,NaN,[],NaN,vk,...,Faberlic,https://faberlic.com/register?sponsor=70025127115,NaN,4,0,0,0.0,NaN,"{'count': 4, 'users': [1467836, 191972714, 220...","{'count': 0, 'users': []}"
13877,49,1436.0,3978922,"Нет ничего дороже слез, что с родных ресничек ...",Female,Natalya_Bulavkina,NaN,Access Denied,NaN,vk,...,СПбИВЭСЭП в Москве,https://www.instagram.com/choco_life7/,28.2,16,0,0,NaN,NaN,"{'count': 16, 'users': [6075010, 6927527, 1087...","{'count': 0, 'users': []}"


In [54]:
clr_data4_from_16396_to_16975=clr_data5.copy()

In [77]:
clr_data4_from_16396_to_16975.to_csv('clr_data_from_16396_to_16975.csv')

In [27]:
clr_data3_till13869=clr_data3.copy()

In [49]:
clr_data4_from_13870_to_16396=clr_data4.copy()

In [48]:
clr_data4.to_csv('clr_data_from_13870_to_16396.csv')

In [57]:
clr_data6.to_csv('cl_data_from_16976_to_18890.csv')

In [61]:
clr_data7.to_csv('cl_data_from_18890_to_19047.csv')

In [66]:
clr_data8.to_csv('cl_data_from_19047_to_20188.csv')

In [69]:
clr_data9.to_csv('cl_data_from_20188_to_20543.csv')

In [11]:
clr_data10.to_csv('cl_data_from_20543_to_21460.csv')

In [13]:
clr_data11.to_csv('cl_data_from_21460_end.csv')

In [10]:
20544+916

21460

In [83]:
new_df.to_csv('to_Levon.csv')

In [82]:
clr_data3.iat()

,post_id,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,date,key_word,likes_count,comments_count,reposts_count,users_liked,users_reposted
0,3056.0,4458562,❄Свободное время для процедуры Ламинирование р...,Female,Tatyana_Demura,NaN,[],NaN,api_android,2018-02-09 15:12:25,#глаза,1,0,0,"{'count': 2, 'users': [92739255, 96650568]}","{'count': 0, 'users': []}"
1,796.0,226768214,#митино#солнце#🌞#в#глаза😜,Female,Ekaterina_Poznyakova,NaN,[],NaN,api_instagram,2018-02-09 15:03:05,#глаза,1,0,0,"{'count': 8, 'users': [313413633, 226768214, 3...","{'count': 0, 'users': []}"
2,1480.0,457471680,Твои #глаза мои #мысли...❤Не ревнуй меня прошу...,Female,Nadya_Dorofeeva,NaN,[],NaN,mvk,2018-02-09 14:52:39,#глаза,3,0,0,"{'count': 3, 'users': [432867540, 193633167, 4...","{'count': 0, 'users': []}"
3,18597.0,182669526,Приватизировала игрушку-подушку. #кошка #чёрна...,Female,Kristina_Osievskaya,NaN,Access Denied,NaN,api_instagram,2018-02-09 14:46:39,#глаза,0,0,0,"{'count': 2, 'users': [132969776, 221483714]}","{'count': 0, 'users': []}"
4,7238.0,2507921,Старые травмы снова себя проявили((<br>Ведь та...,Male,Eduard_Gofman,NaN,"[{'cid': 7239, 'uid': 30498481, 'from_id': 304...",NaN,api_instagram,2018-02-09 14:11:16,#глаза,0,1,0,"{'count': 0, 'users': []}","{'count': 0, 'users': []}"


In [75]:
clr_data3

,post_id,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,date,key_word,likes_count,comments_count,reposts_count,users_liked,users_reposted
0,3056.0,4458562,❄Свободное время для процедуры Ламинирование р...,Female,Tatyana_Demura,NaN,0,NaN,api_android,2018-02-09 15:12:25,#глаза,1,0,0,0,0
1,796.0,226768214,#митино#солнце#🌞#в#глаза😜,Female,Ekaterina_Poznyakova,NaN,0,NaN,api_instagram,2018-02-09 15:03:05,#глаза,1,0,0,0,0
2,1480.0,457471680,Твои #глаза мои #мысли...❤Не ревнуй меня прошу...,Female,Nadya_Dorofeeva,NaN,0,NaN,mvk,2018-02-09 14:52:39,#глаза,3,0,0,0,0
3,18597.0,182669526,Приватизировала игрушку-подушку. #кошка #чёрна...,Female,Kristina_Osievskaya,NaN,0,NaN,api_instagram,2018-02-09 14:46:39,#глаза,0,0,0,0,0
4,7238.0,2507921,Старые травмы снова себя проявили((<br>Ведь та...,Male,Eduard_Gofman,NaN,0,NaN,api_instagram,2018-02-09 14:11:16,#глаза,0,1,0,0,0


In [55]:
clr_data.to_csv('clr_demo.csv')

In [62]:
clr_data

,Name_Surname,comments,comments_count,date,key_word,likes,likes_count,platform,post,post_id,reposts,reposts_count,sex,user_id,users_liked,users_reposted
0,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4458562,"{'count': 2, 'users': [92739255, 96650568]}","{'count': 0, 'users': []}"
1,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,226768214,"{'count': 8, 'users': [313413633, 226768214, 3...","{'count': 0, 'users': []}"
2,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,457471680,"{'count': 3, 'users': [432867540, 193633167, 4...","{'count': 0, 'users': []}"
3,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,182669526,"{'count': 2, 'users': [132969776, 221483714]}","{'count': 0, 'users': []}"
4,NaN,"[{'cid': 7239, 'uid': 30498481, 'from_id': 304...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2507921,"{'count': 0, 'users': []}","{'count': 0, 'users': []}"
5,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6625983,"{'count': 26, 'users': [1202863, 212160494, 18...","{'count': 0, 'users': []}"
6,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18092728,"{'count': 12, 'users': [470533937, 270877288, ...","{'count': 0, 'users': []}"
7,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192590067,"{'count': 29, 'users': [312581184, 456607265, ...","{'count': 0, 'users': []}"
8,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,409278702,"{'count': 0, 'users': []}","{'count': 0, 'users': []}"
9,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174999924,"{'count': 7, 'users': [6827236, 22264496, 1974...","{'count': 0, 'users': []}"


In [66]:
clr_data[clr_data.comments == 'Access Denied']

,Name_Surname,comments,comments_count,date,key_word,likes,likes_count,platform,post,post_id,reposts,reposts_count,sex,user_id,users_liked,users_reposted
3,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,182669526,"{'count': 2, 'users': [132969776, 221483714]}","{'count': 0, 'users': []}"
5,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6625983,"{'count': 26, 'users': [1202863, 212160494, 18...","{'count': 0, 'users': []}"
7,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192590067,"{'count': 29, 'users': [312581184, 456607265, ...","{'count': 0, 'users': []}"
9,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174999924,"{'count': 7, 'users': [6827236, 22264496, 1974...","{'count': 0, 'users': []}"
11,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,328426378,"{'count': 6, 'users': [6952519, 442438297, 433...","{'count': 0, 'users': []}"
12,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,328426378,"{'count': 8, 'users': [53290893, 616297, 15970...","{'count': 0, 'users': []}"
13,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9032358,"{'count': 10, 'users': [19075548, 184904567, 3...","{'count': 0, 'users': []}"
15,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7236446,"{'count': 51, 'users': [21710927, 332096651, 7...","{'count': 0, 'users': []}"
17,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,472430499,"{'count': 0, 'users': []}","{'count': 0, 'users': []}"
18,NaN,Access Denied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1322925,"{'count': 8, 'users': [151479550, 3799950, 326...","{'count': 0, 'users': []}"


In [67]:
new_df.set_value

,post_id,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,date,key_word,likes_count,comments_count,reposts_count
0,3056.0,4458562,❄Свободное время для процедуры Ламинирование р...,Female,Tatyana_Demura,NaN,NaN,NaN,api_android,2018-02-09 15:12:25,#глаза,1,0,0
1,796.0,226768214,#митино#солнце#🌞#в#глаза😜,Female,Ekaterina_Poznyakova,NaN,NaN,NaN,api_instagram,2018-02-09 15:03:05,#глаза,1,0,0
2,1480.0,457471680,Твои #глаза мои #мысли...❤Не ревнуй меня прошу...,Female,Nadya_Dorofeeva,NaN,NaN,NaN,mvk,2018-02-09 14:52:39,#глаза,3,0,0
3,18597.0,182669526,Приватизировала игрушку-подушку. #кошка #чёрна...,Female,Kristina_Osievskaya,NaN,NaN,NaN,api_instagram,2018-02-09 14:46:39,#глаза,0,0,0
4,7238.0,2507921,Старые травмы снова себя проявили((<br>Ведь та...,Male,Eduard_Gofman,NaN,NaN,NaN,api_instagram,2018-02-09 14:11:16,#глаза,0,1,0


In [28]:
i=0
tqdm.pandas(tqdm())
more_new_df = new_df.iloc[100*i,100*(i+1)].progress_apply(commentscrapper,axis = 1)
comments_likes_reposts = more_new_df.apply(pd.Series)


0it [00:00, ?it/s]


IndexError: single positional indexer is out-of-bounds

In [32]:
new_df.iloc[100*i:100*(i+1)]

,post_id,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,date,key_word,likes_count,comments_count,reposts_count
0,3056.0,4458562,❄Свободное время для процедуры Ламинирование р...,Female,Tatyana_Demura,NaN,NaN,NaN,api_android,2018-02-09 15:12:25,#глаза,1,0,0
1,796.0,226768214,#митино#солнце#🌞#в#глаза😜,Female,Ekaterina_Poznyakova,NaN,NaN,NaN,api_instagram,2018-02-09 15:03:05,#глаза,1,0,0
2,1480.0,457471680,Твои #глаза мои #мысли...❤Не ревнуй меня прошу...,Female,Nadya_Dorofeeva,NaN,NaN,NaN,mvk,2018-02-09 14:52:39,#глаза,3,0,0
3,18597.0,182669526,Приватизировала игрушку-подушку. #кошка #чёрна...,Female,Kristina_Osievskaya,NaN,NaN,NaN,api_instagram,2018-02-09 14:46:39,#глаза,0,0,0
4,7238.0,2507921,Старые травмы снова себя проявили((<br>Ведь та...,Male,Eduard_Gofman,NaN,NaN,NaN,api_instagram,2018-02-09 14:11:16,#глаза,0,1,0
5,1157.0,6625983,Быстрая песня на пять восьмых. Сложно. Страшно...,Male,Anatoly_Kozhaev,NaN,NaN,NaN,api_instagram,2018-02-09 13:53:01,#глаза,9,0,0
6,2759.0,18092728,Просто почувствуй себя красивой и желанной!😍🤘<...,Female,Natalya_Voltornist,NaN,NaN,NaN,api_instagram,2018-02-09 13:34:21,#глаза,7,0,0
7,4872.0,192590067,#глаза👸🌋≈тамрекилюбви≈тамбеда,Female,Stanislava_Mirnaya,NaN,NaN,NaN,api_android,2018-02-09 13:20:02,#глаза,15,0,0
8,109.0,409278702,Ваши любимые палетки МАС ❤️<br>В палетку входи...,Female,Margarita_Tarasova,NaN,NaN,NaN,api_iphone,2018-02-09 13:10:08,#глаза,0,0,0
9,49.0,174999924,Видимо не зря в большинстве скульптур изобража...,Male,Vladimir_Zhirkin,NaN,NaN,NaN,api_instagram,2018-02-09 12:26:43,#глаза,4,0,0


In [21]:
likes = api.likes.getList(type='post', item_id = ps, owner_id = us)

VkAPIError: 100. One of the parameters specified was missing or invalid: owner_id not integer. request_params = {'oauth': '1', 'method': 'likes.getList', 'type': 'post', 'item_id': '3056,796,1480,18597,7238', 'owner_id': '4458562,226768214,457471680,182669526,2507921'}

In [ ]:
from tqdm import tqdm, tqdm_pandas
tqdm.pandas(tqdm())
more_new_df = new_df.progress_apply(commentscrapper,axis = 1)
comments_likes_reposts = more_new_df.apply(pd.Series)

In [ ]:
86982/12

In [ ]:
final_data = pd.concat([overall_data,comments_likes_reposts],axis = 1)

In [ ]:
final_data.to_csv(r'D:\vk_all_keywords_365_days_pretty.csv')

In [ ]:
short_df = final_data.copy()

In [ ]:
pd.options.display.max_columns = 45
short_df.head()

In [ ]:
# remove posts with more than 300 characters and posts with no characters
short_df['post']=short_df.text.apply(lambda x: x if len(x)<300 and len(x) !=0 else 'Too Long')

In [ ]:
not_long = short_df[short_df['post']!='Too Long'].reset_index().drop('index', axis = 1)

In [ ]:
import re
list_of_animals = 'кот|кошка|котёнок|котяра|котяры|котяр|котище|котик|собака|пес|собачка|кобель|псина|сука|птица|пташка|птенцы|птенец|щенок'
def find_animal(x):
    my_list = re.findall(list_of_animals,x['post'])
    if len(my_list) == 0:
        return(True)
    else:
        return(False)

In [ ]:
new_df['animal'] = new_df.apply(find_animal,axis = 1)

In [ ]:
(new_df.animal == True).value_counts()

In [ ]:
short_df.to_csv(r'D:\vk_all_keywords_365_days_short.csv')